In [1]:
# testing
'''
from sys import getsizeof
tmp = np.ones((400000, 6000), dtype=np.float32)
getsizeof(tmp)//1024//1024
'''

'\nfrom sys import getsizeof\ntmp = np.ones((400000, 6000), dtype=np.float32)\ngetsizeof(tmp)//1024//1024\n'

In [2]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
import gc
tqdm.pandas()

In [3]:
train = pd.read_pickle('data/train_original.pkl')
test = pd.read_pickle('data/test_original.pkl')
special_cols = [col for col in train.columns if train[col].dtype != np.float64]
feature_cols = [col for col in train.columns if col not in special_cols]

In [4]:
if False:
    def to_texts(row):
        return [str(r).replace("-","_").replace(".","_") for r in row]

    all_df = pd.concat([train[feature_cols], test[feature_cols]], axis=0).reset_index(drop=True)
    all_texts = all_df.progress_apply(to_texts, axis=1)
    all_texts = list(all_texts)
    pd.to_pickle(all_texts, 'cat2vec_data/all_texts.pkl')
else:
    all_texts = pd.read_pickle('cat2vec_data/all_texts.pkl')
    
all_texts[0]

['8_9255',
 '_6_7863',
 '11_9081',
 '5_093',
 '11_4607',
 '_9_2834',
 '5_1187',
 '18_6266',
 '_4_92',
 '5_747000000000001',
 '2_9252',
 '3_1821',
 '14_0137',
 '0_5745',
 '8_7989',
 '14_5691',
 '5_7487',
 '_7_2393',
 '4_284',
 '30_7133',
 '10_535',
 '16_2191',
 '2_5791',
 '2_4716',
 '14_3831',
 '13_4325',
 '_5_1488',
 '_0_4073',
 '4_9306',
 '5_9965',
 '_0_3085',
 '12_9041',
 '_3_8766',
 '16_8911',
 '11_192',
 '10_5785',
 '0_6764',
 '7_8871',
 '4_6667',
 '3_8743',
 '_5_2387',
 '7_3746',
 '11_5767',
 '12_0446',
 '11_6418',
 '_7_017',
 '5_9226',
 '_14_2136',
 '16_0283',
 '5_3253',
 '12_9194',
 '29_046',
 '_0_6940000000000001',
 '5_1736',
 '_0_7474',
 '14_8322',
 '11_2668',
 '5_3822',
 '2_0183',
 '10_1166',
 '16_1828',
 '4_959',
 '2_0771',
 '_0_2154',
 '8_6748',
 '9_5319',
 '5_8056',
 '22_4321',
 '5_0109',
 '_4_7010000000000005',
 '21_6374',
 '0_5663',
 '5_1999',
 '8_86',
 '43_1127',
 '18_3816',
 '_2_344',
 '23_4104',
 '6_5199',
 '12_1983',
 '13_6468',
 '13_8372',
 '1_3675',
 '2_9423',
 '_4

In [5]:
if False:
    from random import shuffle
    from copy import deepcopy as cp

    shuffled_all_texts = cp(all_texts)
    for r in tqdm(shuffled_all_texts):
        shuffle(r)
    pd.to_pickle(shuffled_all_texts, 'cat2vec_data/shuffled_all_texts.pkl')

    shuffled_all_texts[0]

In [6]:
# trained on gcp multi-core server...
'''
from gensim.models.word2vec import Word2Vec
dim = 30
w2v = Word2Vec(shuffled_all_texts, size=dim, window=20, min_count=5, iter=3, sg=1, hs=1, workers=2) # skip-gram
pickle.dump(w2v, open('cat2vec models/st_cat2vec_w2v_md_sg_v1', 'wb'))
'''

"\nfrom gensim.models.word2vec import Word2Vec\ndim = 30\nw2v = Word2Vec(shuffled_all_texts, size=dim, window=20, min_count=5, iter=3, sg=1, hs=1, workers=2) # skip-gram\npickle.dump(w2v, open('cat2vec models/st_cat2vec_w2v_md_sg_v1', 'wb'))\n"

### Prepare cat2vec features

In [7]:
w2v = pickle.load(open('cat2vec models/st_cat2vec_w2v_md_sg_v2', 'rb'))
cat2vec_feats = np.zeros((len(all_texts), 30*len(all_texts[0])), dtype=np.float32)

for i, row in tqdm(enumerate(all_texts)):
    for j, r in enumerate(row):
        if r in w2v.wv:
            cat2vec_feats[i, j*30:(j+1)*30] = w2v.wv[r]

c:\users\khyeh\anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
400000it [04:29, 1484.22it/s]


In [8]:
(cat2vec_feats==0).sum()/(len(all_texts)*30*len(all_texts[0]))

0.005707

In [9]:
if False:
    pd.to_pickle(cat2vec_feats[:train.shape[0]], 'cat2vec_data/cat2vec_feat_train2.pkl')
    pd.to_pickle(cat2vec_feats[train.shape[0]:], 'cat2vec_data/cat2vec_feat_test2.pkl')

In [10]:
# PCA to reduce dimensions
from sklearn.decomposition import PCA

# when copy=False, must use fit transform directly
cat2vec_feats = PCA(n_components=100, svd_solver='arpack', random_state=0, copy=False).fit_transform(cat2vec_feats) 

In [11]:
cat2vec_feats.shape

(400000, 100)

In [12]:
if True:
    pd.to_pickle(cat2vec_feats[:train.shape[0]], 'cat2vec_data/cat2vec_feat_pca_train2.pkl')
    pd.to_pickle(cat2vec_feats[train.shape[0]:], 'cat2vec_data/cat2vec_feat_pca_test2.pkl')